# Custom Loss Functions

In [ ]:

import torch
import torch.nn as nn

class DirectionalLoss(nn.Module):
    def __init__(self):
        super(DirectionalLoss, self).__init__()

    def forward(self, y_pred, y_true):
        direction_true = torch.sign(y_true[1:] - y_true[:-1])
        direction_pred = torch.sign(y_pred[1:] - y_pred[:-1])
        mismatch = direction_true != direction_pred
        return mismatch.float().mean()

class WeightedMAELoss(nn.Module):
    def __init__(self, up_weight=2.0):
        super(WeightedMAELoss, self).__init__()
        self.up_weight = up_weight

    def forward(self, y_pred, y_true):
        weights = torch.where(y_true[1:] > y_true[:-1], self.up_weight, 1.0)
        mae = torch.abs(y_pred - y_true)
        return (mae * weights).mean()

class HybridLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super(HybridLoss, self).__init__()
        self.alpha = alpha

    def forward(self, y_pred, y_true):
        rmse = torch.sqrt(torch.mean((y_pred - y_true) ** 2))
        direction_true = torch.sign(y_true[1:] - y_true[:-1])
        direction_pred = torch.sign(y_pred[1:] - y_pred[:-1])
        direction_loss = (direction_true != direction_pred).float().mean()
        return self.alpha * rmse + (1 - self.alpha) * direction_loss


In [ ]:

y_true = torch.tensor([100.0, 102.0, 101.0, 105.0, 107.0])
y_pred = torch.tensor([99.0, 101.5, 102.0, 104.0, 106.0])

dl = DirectionalLoss()
wmae = WeightedMAELoss(up_weight=2.0)
hl = HybridLoss(alpha=0.7)

print("DirectionalLoss:", dl(y_pred, y_true).item())
print("WeightedMAELoss:", wmae(y_pred, y_true).item())
print("HybridLoss:", hl(y_pred, y_true).item())
